## 1: Overview Of The Data Set

In this project, we'll continue working with the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of facts about countries. The Factbook contains demographic information for each country in the world, including:

- population - The population as of 2015
- population_growth - The annual population growth rate, as a percentage
- area - The total land and water area

You can download the Factbook as a SQLite database [from GitHub](https://github.com/factbook/factbook.sql) if you want to work with it on your own computer. In this guided project, we'll be working with Python and the SQLite command line tool (SQLite Command Shell) to connect to the database, extract data, and perform analysis on the data.

## 2: Overview Of The SQLite Command Shell 

SQLite is a relational database management system that enables us to create databases and query them using SQL syntax. SQLite is simpler than full database systems like MySQL and PostgreSQL. It's good for cases where ease of use is more important than performance. Each SQLite database is stored as a single file, making it easy to transport.

The Factbook database is in the file factbook.db. Thedb at the end is a file extension that's short for database.

We can open the Factbook database in the SQLite Command Shell by navigating to the same folder as factbook.db, then typing sqlite3 factbook.db on the command line. This enables us to manage the database and run SQL queries.

### Instructions:
Use the SQLite Command Shell to explore factbook.db.

- Connect to factbook.db using the SQLite Command Shell.
- Type .help into the shell to see a list of commands you can run in the shell.
- Type .tables to see a list of the tables in the database.
- If you type .header on, you'll see the column headers when you run queries.
If you type an incomplete command (one that's missing the ending semicolon, for example), SQLite won't execute it. Instead, it will take you to an indented line. Type the semicolon (;) in the indented line (and press Enter) to exit the indentation and run the command. Here's an example:

<img src="https://dq-content.s3.amazonaws.com/nPRG1kN.png">

When you're done with the SQLite Command Shell, type .quit. Don't quit the shell just yet, though, because you'll be using it in the next step.

In [157]:
%%bash
cd data/
#sqlite3 factbook.db

# since this is an interactive command-line program, I can't show everything in this NB

# In later steps I'll just write python code instead of using command-line sqlite3+scripts

## 3: Running Queries In The SQLite Command Shell
The SQLite Command Shell also allows us to run any valid SQL query. For example, we could run the following:

``SELECT * FROM facts;``

This will show us all of the rows in the facts table.

### Instructions:

Run some queries in the SQLite Command Shell. Be sure to enter .header on to see the headers for each column.
While you should think of your own queries, here are a couple of examples:

- SELECT * FROM facts ORDER BY population DESC LIMIT 10;
- SELECT * FROM facts ORDER BY area_land ASC LIMIT 10;

You may notice that these queries return some strange results, such as Ethiopia having the least land area. The queries also include non-national entities like the European Union and Akrotiri.
The data is fairly messy, and some values in the area_land column are missing. Add WHERE area_land != "" to the query before the ORDER BY clause to remove the invalid rows. You may also need to try additional filtering.
When you're done exploring, you can quit the SQLite Command Shell with .quit.

## 4: Using Python With SQLite

The sqlite3 library, which comes with Python by default, allows us to connect to SQLite databases. To do this, we open a database connection, then create an object that can run queries.

For example, this code will let us connect to factbook.db and select all of the rows:


``import sqlite3
conn = sqlite3.connect('factbook.db')
​
c = conn.cursor()
c.execute('SELECT * FROM facts;')
​
print(c.fetchall())``

The code above creates a Connection object. We then create a Cursor instance, which can execute queries. Finally, we execute a query and display the results using the print function. To learn more about the sqlite3 library, read the package documentation on the official Python website.

### Instructions:

- Write code in query.py that selects the 10 least populated countries from the facts table, and then prints the results.
- Execute query.py from the command line by running python query.py.

## 5: Computing Population Projections

You can read the results of a SQL query into a pandas dataframe using the read_sql_query function, which the official pandas website documents. The read_sql_query function takes a SQL query string and a connection object, and returns a dataframe containing all of the rows and columns from the query.

When pandas reads in columns this way, it automatically uses the column types from the original the database. Blank entries in the database (like the ones in the area_land column) will have NaN values in a dataframe, which means "Not a Number." This is because pandas can't have blanks in numeric columns; it uses NaN to signify invalid or missing values instead.

You can learn more about how to work with missing data in the pandas documentation. For now, we'll just use the dropna method with the axis=0 argument, which will drop any rows that have NaN values.

### Instructions:

Read the facts table into pandas, and then compute the projected population for each country in 2050. Here's a rough outline of the steps you'll need to take:

- Create a script called growth.py.
- Read facts into a pandas dataframe using read_sql_query.
- Filter out any rows that have invalid data. Look for things like the value 0 in the area_land column, which doesn't make sense.
- Write a function that takes in a country's initial population and growth rate, and then outputs the final population. The annual population growth (expressed as a percentage) for each country is in the population_growth column, while the initial population is in the population column.
    - The formula for compound annual population growth is:
    
N=N0e(rt)  
N is the final population, N0 is the initial population, e is a constant value we can access with math.e, r is the rate of annual change, expressed as a decimal (so 1.5 percent should be .015), and t is the number of years we want to project out. Assume that you'll be starting in January 2015 and ending in January 2050, covering a period of 35 years. For example, let's say you have a country with 5000 people and a 4 percent annual growth rate. The formula would be N=5000∗e(.04∗35). Use the apply method on pandas dataframes to compute what the population will be in 2050 for each row in the data. Use the dataframe sort_values method to sort on the 2050 population in descending order. Print the 10 countries that will have the highest populations in 2050.

In [5]:
# example
import sqlite3

conn = sqlite3.connect("data/factbook.db")

query = "select name from facts order by population asc limit 10"
print(conn.execute(query).fetchall())

[('Ashmore and Cartier Islands',), ('Coral Sea Islands',), ('Heard Island and McDonald Islands',), ('Clipperton Island',), ('French Southern and Antarctic Lands',), ('Bouvet Island',), ('Jan Mayen',), ('British Indian Ocean Territory',), ('South Georgia and South Sandwich Islands',), ('Navassa Island',)]


#### Instructions answer (using df.apply(lambda x: ... ))

In [158]:
# imports
import sqlite3 as sql
import pandas as pd
import math # for math.e

# connect to sqlite3 db
conn = sql.connect("data/factbook.db")

# write query
query = "select * from facts"

# read query into pandas
df = pd.read_sql_query(query, conn)

# check df size
print(df.shape)
# see how many null values for each column and total
nulls = df.isnull().sum(axis=0)
print(nulls)
print(sum(nulls))
# drop NaNs
df.dropna(axis=0, inplace=True)
nulls = df.isnull().sum(axis=0)
print(nulls)
# filter multiple columns for bad data
df = df[(df.area>0) & (df.birth_rate>0) & (df.death_rate>0) & (df.name != '')]
# check new df size
print(df.shape)

# apply lambda function to calculate projected population and create new col
df["2050_projected_pop"] = df.apply(lambda x: round(x["population"] * 
    pow(math.e,x["population_growth"])*0.01*35), axis = 1)

# sort by 2050 projected population, descending order, print top 10
print(df.sort_values(by="2050_projected_pop", ascending=False)[:10]['name'])

(261, 13)
id                    0
code                  0
name                  0
area                 12
area_land            15
area_water           18
population           19
population_growth    25
birth_rate           33
death_rate           33
migration_rate       38
created_at            0
updated_at            0
dtype: int64
193
id                   0
code                 0
name                 0
area                 0
area_land            0
area_water           0
population           0
population_growth    0
birth_rate           0
death_rate           0
migration_rate       0
created_at           0
updated_at           0
dtype: int64
(215, 13)
76                                 India
36                                 China
128                              Nigeria
181                               Uganda
39     Congo, Democratic Republic of the
131                             Pakistan
13                            Bangladesh
171                             Tanzania
185        

In [ ]:
# Math library for math.e
import math

# apply lambda function to calculate projected population and create new col
df["2050_projected_pop"] = df.apply(lambda x: round(x["population"] * 
    pow(math.e,x["population_growth"])*0.01*35), axis = 1)

# sort by 2050 projected population, descending order, print top 10
print(df.sort_values(by="2050_projected_pop", ascending=False)[:10]['name'])

## 6: Summing Columns To Compute Total Area

We can add up all of the values in a column by using the SUM function in a SQL query. For example, we can calculate the total population with this query:


``SELECT SUM(population) from facts;``

We can also add a WHERE clause, like this:

``SELECT SUM(population) from facts WHERE area_land != "";``

### Instructions: 

Use SQL and Python to compute the ratio of land area to water area that each country claims. Here's a rough outline of the steps you'll need to take:

- Write a script called area.py.
- Query to get the total of the area_land column.
- Query to get the total of the area_water column.
- Divide area_land by area_water and print the result.

In [152]:
import sqlite3
import pandas as pd

# connect to sqlite3 db
conn = sqlite3.connect("data/factbook.db")

# query to get sums of columns
area_land_total = pd.read_sql_query("select sum(area_land) from facts", conn).iloc[0][0]
area_water_total = pd.read_sql_query("select sum(area_water) from facts", conn).iloc[0][0]

# calculate land-to-water ratio
land_to_water_ratio = area_land_total / area_water_total
print(land_to_water_ratio)

27.7515734041
